In [1]:
from dataclasses import dataclass
from typing import Optional, Tuple, Union, TYPE_CHECKING, List, Dict, Any
import re

import lmfit as lm
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from dat_analysis.analysis_tools.new_procedures import Process, DataPlotter, PlottableData
from dat_analysis.analysis_tools.general_fitting import calculate_fit, FitInfo
from dat_analysis.new_dat.dat_hdf import get_dat
import dat_analysis.useful_functions as U

if TYPE_CHECKING:
    import h5py

In [2]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
# pio.renderers.default = 'browser'

In [3]:
def add_waterfall(fig, x, data):
    for i, d in enumerate(data):
        d = U.decimate(d, dat.Logs.measure_freq, numpnts=300, ntaps=100)
        x = U.get_matching_x(x, d)
        fig.add_trace(go.Scatter(x=x, y=d, mode='lines', name=f'row{i}'))
    return fig

def full_plot(datnum):
    dat = get_dat(datnum)
    fig = go.Figure()
    if 'wave0_2d' in dat.Data.data_keys:
        data = dat.Data.get_data('wave0_2d')
    elif 'cscurrent_2d' in dat.Data.data_keys:
        data = dat.Data.get_data('cscurrent_2d')
    elif 'current_2d' in dat.Data.data_keys:
        data = dat.Data.get_data('current_2d')
    else:
        raise FileNotFoundError(f'Valid data keys are {dat.Data.data_keys}')
        
    fig = add_waterfall(fig, dat.Data.x, data)
    with dat.hdf_read as f:
        scan_duration = f['Logs']['General'].attrs['time_elapsed']
    fig.update_layout(title=f'Dat{dat.datnum}: Total scan time: {scan_duration:.1f}s', xaxis_title=dat.Logs.x_label, yaxis_title='Current /nA')
    return fig

def resample(x, data, pts=200):
    data = U.decimate(data, 1000, numpnts=pts, ntaps=100)
    x = U.get_matching_x(x, data)
    return x, data


In [4]:
dat = get_dat(156)
fig = go.Figure()
fig = add_waterfall(fig, dat.Data.x, dat.Data.get_data('wave0_2d'))
fig.update_layout(title=f'Dat{dat.datnum}', xaxis_title='Gates /mV', yaxis_title='Current /nA')

In [5]:
for num in [156, 157, 158, 159, 173]:
    fig = full_plot(num)
    fig.show()

In [6]:
fig = full_plot(159)
dat = get_dat(176)
x, data = dat.Data.x, dat.Data.get_data('cscurrent')
x, data = resample(x, data)
fig.add_trace(go.Scatter(x=x, y=data, name=f'Dat{dat.datnum}'))
fig


In [7]:
fig = full_plot(178)
fig

In [8]:
fig2 = full_plot(179)
fig2

In [9]:
for d in fig2.data:
    fig.add_trace(d)
fig

In [10]:
(1, 2) + (3, 4)

(1, 2, 3, 4)

In [11]:
fig1 = full_plot(156)
fig2 = full_plot(181)
for d in fig2.data:
    d['name'] = f'dat181{d["name"]}'
    fig1.add_trace(d)
fig1

In [12]:
fig = full_plot(180)

In [13]:
fig

In [29]:
datnums = range(220, 231+1)
# datnums = range(307, 318+1)


fig = go.Figure()
for num in datnums:
    dat = get_dat(num)
    x = dat.Data.x
    data = dat.Data.get_data('csurrent_2d') if 'csurrent_2d' in dat.Data.data_keys else dat.Data.get_data('cscurrent_2d')
#     data = dat.Data.get_data('current_2d')
    data = np.nanmean(data, axis=0)
    data, x = U.resample_data(data, x=x, max_num_pnts=300)
    fig.add_trace(go.Scatter(x=x, y=data, name=f'Dat{dat.datnum}: {min(x):.0f}'))
fig.update_layout(
    legend=dict(title='Max sweep /mV'),
    xaxis_title=dat.Logs.x_label,
    yaxis_title='Current /nA',
    title='Stability of Charge Sensor QPC (Bale)'
#     title='Stability of Charge Sensor QPC (Dendi)'
#     title='Leakage of Charge Sensor QPC (Bale)'
#     title='Leakage of Charge Sensor QPC (Dendi)'
)

fig
 
    

In [32]:
# datnums = range(15, 19+1)
# datnums = range(20, 24+1)
datnums = range(25, 29+1)


fig = go.Figure()
for num in datnums:
    dat = get_dat(num, host_name='qdev-dunker', user_name='Tim', experiment_name='20220722_CPH1_CPH2_stability')
    x = dat.Data.x
    data = dat.Data.get_data('current_2d')
    data = np.nanmean(data, axis=0)
    data, x = U.resample_data(data, x=x, max_num_pnts=300)
    fig.add_trace(go.Scatter(x=x, y=data, name=f'Dat{dat.datnum}: {min(x):.0f}'))
fig.update_layout(
    legend=dict(title='Max sweep /mV'),
    xaxis_title=dat.Logs.x_label,
    yaxis_title='Current /nA',
    title='QPC Stability'
)

fig
 